In [1]:
%cd ..
import numpy as np
import pandas as pd
import src.fetch as fetch

/home/cole/Documents/BDS/bds-final-project


In [2]:
restaurant_insp = fetch.fetch_restaurant_inspection_data()
insp = fetch.fetch_inspection_data(True)

In [34]:
hard = insp[['business_name', 'is_closed']]

In [14]:
# Top keywords for closures
kw = {}
names = list(hard.loc[hard['is_closed'] == 1]['business_name'])
for n in names:
    s = str(n).lower().strip().split()
    for ss in s:
        if ss not in kw:
            kw[ss] = 1
        else:
            kw[ss] = kw[ss] + 1

In [23]:
top_for_closed = sorted(kw, reverse=True, key=kw.get)[:15]
top_for_closed

['inc',
 'llc',
 'corp',
 'restaurant',
 'cafe',
 'rest',
 'avenue',
 'pizza',
 'group',
 'bar',
 'and',
 'food',
 'ltd',
 'corporation',
 'new']

In [24]:
# Top keywords for non-closures
kw = {}
names = list(hard.loc[hard['is_closed'] == 0]['business_name'])
for n in names:
    s = str(n).lower().strip().split()
    for ss in s:
        if ss not in kw:
            kw[ss] = 1
        else:
            kw[ss] = kw[ss] + 1

In [22]:
top_for_open = sorted(kw, reverse=True, key=kw.get)[:15]
top_for_open

['inc',
 'llc',
 'corp',
 'restaurant',
 'cafe',
 'rest',
 'avenue',
 'pizza',
 'group',
 'bar']

In [58]:
union = list(set(top_for_closed + top_for_open))
union

['and', 'food', 'ltd', 'corporation', 'new']

In [59]:
for c in union:
    hard[c] = np.where(hard['business_name'].str.lower().str.contains(c), 1, 0)

/home/cole/.anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(hard.drop(['is_closed', 'business_name'], 1), hard['is_closed'])

In [61]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200)
gbm.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [62]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, gbm.predict(x_test))

0.0